<a href="https://colab.research.google.com/github/anikap1234/zillow-web-scraper/blob/main/zillow_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install bs4
from bs4 import BeautifulSoup
import json
import time
import csv

In [2]:
import requests
class ZillowScraper():
    results = []
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'cookie': 'zguid=23|%24d14f0059-d03b-4422-9f57-5862fcd13490; _ga=GA1.2.1741285320.1590755697; zjs_user_id=null; zjs_anonymous_id=%22d14f0059-d03b-4422-9f57-5862fcd13490%22; __gads=ID=1050523ba93d593d:T=1590755700:S=ALNI_MZlJJ_xqSbd51oJisV_HY4g017Ehw; _gcl_au=1.1.2000298647.1590755705; KruxPixel=true; _fbp=fb.1.1590755705919.1815197270; _pxvid=d6c5ec75-a1a8-11ea-b8a9-0242ac120009; KruxAddition=true; JSESSIONID=3E7EBDB1F8931DF7D0DE9992546AE0B3; zgsession=1|200e23e0-9534-4d27-931f-caa3de6b483b; _gid=GA1.2.1328942480.1590858452; _gat=1; DoubleClickSession=true; GASession=true; _uetsid=fdde22d5-862a-8a7d-93e4-a16c574edf91; _pin_unauth=YzUyOGQ2OGMtMmQ3YS00NGZkLTg3MmEtOGJlODM1YWMwMTA1; _px3=026336d3721eec42bcdec3278ad2d3ac2014d5e65707b21624fb2e743d9a89be:mq3WRz2RNL5PBIvbYNHCxq5VfXHXy2YKC+8Lqn97pIw8MiKppH7Cx7AjKzbAFi1zcehKGY36aIgsnE9NiPKwlw==:1000:4U1o3ogIQ0KzfyMd2QYEFGDnD1augezy5bJlzEn9ZHE89B2uEIxDg8BmsGj8szPwyIz1Yv15S2V0TV5P+0jCFisfGk92XM4DM7K13GCtNr0HXhNGftVBFxVrCv8ApRphw/Qwj7AcagCh9i6FPiQGLFruxVASJXLsNpFeWimekVY=; AWSALB=ZKAGBcH2BwM6D1bRKOPynbOqyclySGz5U/fZB+wO3MYQ91UR9A5rFVtFsmjOkrMASUJguhtsJRZDM7IlBiWVT/pGw2S0BjxgEZmpFPrBZEqU2lWTE2NMArtecZD2; AWSALBCORS=ZKAGBcH2BwM6D1bRKOPynbOqyclySGz5U/fZB+wO3MYQ91UR9A5rFVtFsmjOkrMASUJguhtsJRZDM7IlBiWVT/pGw2S0BjxgEZmpFPrBZEqU2lWTE2NMArtecZD2; search=6|1593450465587%7Crect%3D40.843698984643765%252C-73.50417109960938%252C40.567821651427245%252C-74.45174190039063%26rid%3D6181%26disp%3Dmap%26mdm%3Dauto%26p%3D2%26z%3D0%26lt%3Dfsbo%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%096181%09%09%09%09%09%09',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }
    def fetch(self, url, params):
        response = requests.get(url, headers=self.headers, params=params)
        print(response.status_code)
        return response

    def parse(self, response):
        content = BeautifulSoup(response, 'lxml')
        deck = content.find('ul', {'class': 'photo-cards'})
        print(deck)
        if deck:
          for card in deck:
              script = card.find('script', {'type': 'application/ld+json'})
              if script:
                  print(script.prettify())
                  script_json = json.loads(script.contents[0])

                  self.results.append({
                      'latitude': script_json['geo']['latitude'],
                      'longitude': script_json['geo']['longitude'],
                      'floorSize': script_json['floorSize']['value'] if script_json["floorSize"].get('value') else "",
                      'address': script_json['address']['streetAddress'],
                      'address_Locality': script_json['address']['addressLocality'],
                      'address_Region': script_json['address']['addressRegion'],
                      'address_postalCode': script_json['address']['postalCode'],
                      'url': script_json['url'],
                                    })
    def to_csv(self):
        with open('zillow.csv', 'w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys())
            writer.writeheader()

            for row in self.results:
                writer.writerow(row)
    def run(self):
        url = 'https://www.zillow.com/new-york-ny/fsbo/'

        for page in range(1, 13):
            params = {
                'searchQueryState': '{"pagination":{"currentPage": %s},"mapBounds":{"west":-74.40093013281245,"east":-73.55498286718745,"south":40.4487909557045,"north":40.96202658306895},"regionSelection":[{"regionId":6181,"regionType":6}],"isMapVisible":false,"filterState":{"isForSaleByAgent":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false}},"isListVisible":true}' %page
            }
            res = self.fetch(url, params)
            self.parse(res.text)
            time.sleep(2)
        self.to_csv()

In [3]:
if __name__ == '__main__':
  scraper = ZillowScraper()
  scraper.run()

200
<ul class="List-c11n-8-100-4__sc-1smrmqp-0 StyledSearchListWrapper-srp-8-100-4__sc-1ieen0c-0 cRHZNY laYCaY photo-cards"><li class="ListItem-c11n-8-100-4__sc-13rwu5a-0 StyledListCardWrapper-srp-8-100-4__sc-wtsrtn-0 ggnGBj iXHkOx"><div data-renderstrat="ssr" style="height:100%;display:grid"><div><script type="application/ld+json">{"@type":"SingleFamilyResidence","@context":"http://schema.org","name":"8804 195th St, Jamaica, NY 11423","floorSize":{"@type":"QuantitativeValue","@context":"http://schema.org","value":"2,208"},"address":{"@type":"PostalAddress","@context":"http://schema.org","streetAddress":"8804 195th St","addressLocality":"Jamaica","addressRegion":"NY","postalCode":"11423"},"geo":{"@type":"GeoCoordinates","@context":"http://schema.org","latitude":40.716698,"longitude":-73.76834},"url":"https://www.zillow.com/homedetails/8804-195th-St-Jamaica-NY-11423/32144590_zpid/"}</script><article class="StyledPropertyCard-c11n-8-100-4__sc-g2ckw9-0 hocnjx StyledPropertyCard-srp-8-100-